# Submitting Qiskit Circuits to Azure Quantum with the QDK

This notebook shows how to take an arbitrary Qiskit `QuantumCircuit`, compile it to QIR using the Microsoft Quantum Development Kit (QDK) Python APIs, and submit it as a job to an Azure Quantum target. It also demonstrates how to handle parameterized circuits by binding parameters before submission.

The workflow demonstrated here:

1. Build (or load) a Qiskit `QuantumCircuit`.
2. Convert it to OpenQASM 3 text (`qiskit.qasm3.dumps`).
3. Compile that OpenQASM 3 source to QIR with `qdk.openqasm.compile`.
4. Create (or attach to) an Azure Quantum workspace with `qdk.azure.Workspace`.
5. Select a target (e.g. a simulator such as `rigetti.sim.qvm`).
6. Submit the QIR payload and retrieve measurement results.

Why use the QDK in this flow?

- Local compilation: QIR generation happens locally—no external CLI needed.
- Consistency: The same compiler stack powers Q#, OpenQASM, and (via translation) Qiskit interoperability.

This notebook focuses strictly on submission. For local resource estimation of Qiskit circuits, see the separate estimation sample.

## Prerequisites

This notebook assumes the `qdk`, Azure Quantum integration, and Qiskit packages are installed. You can install everything (including the Azure + Qiskit extras) with:

In [ ]:
%pip install qdk[azure,qiskit]

This installs:
- The base qdk package (compiler, OpenQASM/QIR tooling)
- Azure Quantum client dependencies for submission
- Qiskit for circuit construction

After installing, restart the notebook kernel if it was already running. You can verify installation with:

In [ ]:
import qiskit, qdk, qdk.azure  # should import without errors

## Submitting a simple Qiskit circuit

We start with a minimal circuit that prepares single-qubit superpositions on two qubits and measures them. After constructing the circuit we’ll submit it to an Azure Quantum target.

In [ ]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2, 2)

circuit.h(0)
circuit.measure(0, 0)

circuit.h(1)
circuit.measure(1, 1)

circuit.draw(output="text")

## Configure Azure Quantum workspace connection

To connect to an Azure workspace replace the following variables with your own values.

In [ ]:
subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'myresourcegroup'
workspace_name = 'myworkspace'
location = 'westus'

In [ ]:
from qiskit import qasm3
from qdk.openqasm import compile
from qdk.azure import Workspace
from qdk import TargetProfile

def submit_qiskit_circuit_to_azure(circuit, target_name, name, shots=100):
    qasm3_str = qasm3.dumps(circuit)
    qir = compile(qasm3_str, target_profile=TargetProfile.Base)

    workspace = Workspace(
        subscription_id=subscription_id,
        resource_group=resource_group,
        name=workspace_name,
        location=location,
    )
    target = workspace.get_targets(target_name)
    job = target.submit(qir, name, shots=shots)
    return job.get_results()

results = submit_qiskit_circuit_to_azure(circuit, "rigetti.sim.qvm", "qiskit-job")
print(results)

## Parameterized Qiskit circuits

Many algorithms use symbolic parameters. Before compiling to QIR and submitting, all circuit parameters must be bound to numeric values. Below we build a simple entangling ladder, apply a rotation parameterized by θ across all qubits, then uncompute the entanglement and measure the first qubit.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter


def get_parameterized_circuit(n = 5) -> QuantumCircuit:
    theta = Parameter("θ")
    qc = QuantumCircuit(n, 1)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
    qc.rz(theta, range(n))

    for i in reversed(range(n - 1)):
        qc.cx(i, i + 1)
    qc.h(0)
    qc.measure(0, 0)
    return qc

# Build the symbolic (parameterized) circuit
circuit = get_parameterized_circuit()

# Bind θ to a numeric value, then visualize the bound circuit
circuit.assign_parameters({"θ": 0.5}).draw(output="text")

## Submitting a bound parameterized circuit

Below we reuse the parameterized circuit from above, bind θ to 0.5, and submit the resulting (fully concrete) circuit to the Azure Quantum target using the same helper function as before. The printed result shows the measurement counts. Change the value of θ (or loop over several values) to explore how the outcome distribution varies.

In [ ]:
# Reuse the previously defined parameterized circuit
# (circuit is symbolic and must have θ bound to a value before submission)

results = submit_qiskit_circuit_to_azure(
    circuit.assign_parameters({"θ": 0.5}),
    "rigetti.sim.qvm",
    "qiskit-parameterized-job"
)
print(results)